In [9]:
# load the logs and binary files for prediction
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

In [10]:
# load the sscaler, one hot encoder, model and label encoder

from tensorflow.keras.models import load_model

model = load_model('model.h5', custom_objects={"InputLayer": tf.keras.layers.Input})

with open('standard_scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('onehotencoder_geography.pkl', 'rb') as f:
    one_hot_encoder_geo = pickle.load(f)


with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

In [16]:
#eg. input data

input_data={
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

input_df=pd.DataFrame([input_data])

In [18]:
geo_encoded = one_hot_encoder_geo.transform([input_df['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=['Geography_France', 'Geography_Germany', 'Geography_Spain'])
input_df = pd.concat([input_df.drop(columns='Geography'), geo_encoded_df], axis=1)

C:\Users\hp\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [19]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])

In [20]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [21]:
# perform Standard scaler

input_df = scaler.transform(input_df)
input_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [22]:
# predict for input

predictions = model.predict(input_df)
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 500ms/step


array([[0.06657011]], dtype=float32)

In [24]:
prediction_pronb = predictions[0][0]

In [25]:
if prediction_pronb > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
